In [288]:
import numpy as np
from jplephem.spk import SPK
from astropy import units as u
from mpcpp import MPC_library as mpc

In [419]:
rot_mat = mpc.rotate_matrix(-mpc.Constants.ecl*0.99999999)
observatories = mpc.Observatory()

In [420]:
helio_OBS_equ = observatories.getObservatoryPosition(obsCode='500', jd_utc=2458892.0, old=False) * u.au
print(helio_OBS_equ)

[-0.78780537  0.54557587  0.23650568] AU


In [421]:
helio_OBS_ecl = np.dot(rot_mat, helio_OBS_equ.reshape(-1, 1)).flatten()
print(helio_OBS_ecl)

[-7.87805371e-01  5.94632628e-01 -2.77886136e-05] AU


In [422]:
horizons_equ = np.array([7.878053705097503E-01, -5.455758662127957E-01, -2.365056822060531E-01]) * u.au
horizons_ecl = np.array([7.878053705097503E-01, -5.946326283777568E-01, 2.789540572789949E-05]) * u.au
print((horizons_equ + helio_OBS_equ).to(u.m))
print((horizons_ecl + helio_OBS_ecl).to(u.m))


[ 0.00111278 -0.01551253 -0.00725385] m
[1.11278290e-03 7.30899063e-01 1.59758766e+04] m


In [477]:
rot_mat = mpc.rotate_matrix(-84381.448*(1./3600)*np.pi/180.)
horizons_equ = np.array([7.878053705097503E-01, -5.455758662127957E-01, -2.365056822060531E-01]) * u.au
horizons_ecl = np.array([7.878053705097503E-01, -5.946326283777568E-01, 2.789540572789949E-05]) * u.au
horizons_rot2ecl = np.dot(rot_mat, horizons_equ.reshape(-1, 1)).flatten()
print(horizons_ecl)
print(horizons_rot2ecl)
print((horizons_ecl - horizons_rot2ecl).to(u.m))


[ 7.87805371e-01 -5.94632628e-01  2.78954057e-05] AU
[ 7.87805371e-01 -5.94632628e-01  2.78954057e-05] AU
[ 0.0000000e+00  0.0000000e+00 -5.2064382e-06] m


In [446]:
def stuff(horizons_equ, horizons_ecl, scale):
    rot_mat = mpc.rotate_matrix(-mpc.Constants.ecl*scale)
    horizons_rot2ecl = np.dot(rot_mat, horizons_equ.reshape(-1, 1)).flatten()
    return horizons_rot2ecl-horizons_ecl

In [475]:
scales= np.arange(0.999999,1.000001,0.00000001)
foo = []
for scale in scales:
    foo.append(stuff(horizons_equ, horizons_ecl, scale))

In [481]:
foo[np.argmin(np.abs(np.array(foo)[:,2]))].to(u.m)
print(84381.4118*scales[np.argmin(np.abs(np.array(foo)[:,2]))])

84381.44808400769


In [515]:
list_of_observations = ['     0000109  C2010 09 01.17463 21 47 39.44 -07 20 06.8          24.2 Rr     644',
                        '     0000109  C2010 09 01.21844 21 47 36.69 -07 20 04.5          21.6 Rr     644',
                        '     0000164*`C2010 10 23.36013603 36 52.36 +39 21 49.7          19.4 J      568',
                        '     0000164 `C2010 10 23.37185303 36 51.66 +39 21 56.3          19.4 J      568',
                        '     0000164 `C2010 10 23.38293503 36 50.97 +39 22 02.6          19.4 J      568',
                        '     0000164 `C2010 10 23.39401703 36 50.31 +39 22 08.8          19.2 J      568',
                        '     0000180*`C2008 03 07.21053207 03 42.69 +12 30 01.4          20.7 J      568',
                        '     0000180 `C2008 03 07.22220307 03 42.08 +12 29 59.4          20.2 J      568',
                        '     0000180 `C2008 03 07.23339507 03 41.32 +12 29 59.4          22.6 J      568']

In [549]:
list_of_lists = [[list_of_observations[0]]]
for obs in list_of_observations[1:]:
    if obs[:12] == list_of_lists[-1][-1][:12]:
        list_of_lists[-1].append(obs)
    else:
        list_of_lists.append([obs])

In [550]:
print(list_of_lists)

[['     0000109  C2010 09 01.17463 21 47 39.44 -07 20 06.8          24.2 Rr     644', '     0000109  C2010 09 01.21844 21 47 36.69 -07 20 04.5          21.6 Rr     644'], ['     0000164*`C2010 10 23.36013603 36 52.36 +39 21 49.7          19.4 J      568', '     0000164 `C2010 10 23.37185303 36 51.66 +39 21 56.3          19.4 J      568', '     0000164 `C2010 10 23.38293503 36 50.97 +39 22 02.6          19.4 J      568', '     0000164 `C2010 10 23.39401703 36 50.31 +39 22 08.8          19.2 J      568'], ['     0000180*`C2008 03 07.21053207 03 42.69 +12 30 01.4          20.7 J      568', '     0000180 `C2008 03 07.22220307 03 42.08 +12 29 59.4          20.2 J      568', '     0000180 `C2008 03 07.23339507 03 41.32 +12 29 59.4          22.6 J      568']]


In [551]:
np.shape(list_of_lists)

(3,)

In [585]:
from sifter import precalc as p
from importlib import reload
reload(p)

<module 'sifter.precalc' from '/home/malexand/.anaconda3/envs/astroconda36/lib/python3.6/site-packages/sifter/precalc.py'>

In [586]:
t=p.Tracklets()

In [587]:
t.parse_all_observations(list_of_observations)

Received more than 2 observations.
Only using first and last one.
Received more than 2 observations.
Only using first and last one.


[{'observations': ['     0000109  C2010 09 01.17463 21 47 39.44 -07 20 06.8          24.2 Rr     644',
   '     0000109  C2010 09 01.21844 21 47 36.69 -07 20 04.5          21.6 Rr     644'],
  'JD': 2455441,
  'JD2': 2455441,
  'RoM': <Quantity 0.25981539 deg / d>,
  'AoM': <Angle 4.76853472 rad>,
  'HP': 1184,
  'HP2': 1184,
  'tracklet_name': '0000109_2455440.67463__644',
  'obs_helio-ecliptic_xyz': [<Quantity [ 9.38969202e-01, -3.70101071e-01,  3.85420707e-05] AU>,
   <Quantity [ 9.39242293e-01, -3.69399022e-01,  3.69512015e-05] AU>]},
 {'observations': ['     0000164*`C2010 10 23.36013603 36 52.36 +39 21 49.7          19.4 J      568',
   '     0000164 `C2010 10 23.37185303 36 51.66 +39 21 56.3          19.4 J      568',
   '     0000164 `C2010 10 23.38293503 36 50.97 +39 22 02.6          19.4 J      568',
   '     0000164 `C2010 10 23.39401703 36 50.31 +39 22 08.8          19.2 J      568'],
  'JD': 2455493,
  'JD2': 2455493,
  'RoM': <Quantity 0.25001984 deg / d>,
  'AoM': <Angle